#### Интегральное уравнение Фредгольма: $u(x) - 0.1\int_{0}^{1}\frac{1}{10 - xy}u(y)dy = 1 + x^2$

#### Интегральное уравнения Вольтерра: $u(x) - \int_{0}^{x}sin(x-y)u(y)dy = 1 - cos(x);  
x\in[0, \frac{\pi}{2}]$
 
* Методом механических квадратур найти решение интегрального уравнения Фредгольма. Для вычисления интеграла использовать квадратурную формулу Симпсона.
* Методом замены ядра на вырожденное найти решение интегрального уравнения Фредгольма. 
* Методом последовательных приближений найти решение интегрального уравнения Фредгольма с точностью <b>ε = 0.001</b>
* Методом механических квадратур найти решение интегрального уравнения Вольтерра. Для вычисления интеграла использовать квадратурную формулу трапеции.


In [1]:
import numpy as np
from math import cos, sin, pi
import scipy.integrate
import scipy.interpolate
N = 11
a, h, b = 0.0, 0.1, 1
X = [round(a + i * h, 2) for i in range(N)]
K = lambda x, y: 1 / (10 - x*y)
f = lambda x: 1 + x**2
L = 0.1
EPS = 10**-3

In [2]:
def integral(func_1, func_2):
    func = lambda x: func_1(x) * func_2(x)
    I = scipy.integrate.quad(func, a, b)
    return I[0]

In [3]:
def kernel_change():
    alpha = {
        0: lambda x: (2 * x - 1) * (x - 1),
        1: lambda x: -4 * x * (x - 1),
        2: lambda x: x * (2 * x - 1)
        }
    beta = {
        0: lambda y: 0.1,
        1: lambda y: 2 / (20 - y),
        2: lambda y: 1 / (10 - y)
        }
    B = np.array( [integral(beta[i], f) for i in range(3)] )
    M = np.zeros((3,3))
    for i in range(3):
        for j in range(3):
            if i == j:
                M[i][i] = 1 - L*integral(alpha[i],beta[i])
            else:
                M[i][j] = -L*integral(alpha[j], beta[i])
    C = np.linalg.solve(M,B)
    u = lambda x: f(x) + L*sum(C[i]*alpha[i](x) for i in range(3))
    for i in range(N):
        print("{}\t{}\n".format(X[i], u(X[i])), end='')

In [4]:
def mech_quadr_Fredholm():
    A = [2 * h / 3 for i in range(N)]
    A[0] = A[-1] = h / 3
    for i in range(1, N, 2):
        A[i] *= 2

    g = [f(point) for point in X]
    M = np.zeros((N, N))
    for i in range(N):
        M[i][i] = 1 - L * A[i] * K(X[i], X[i])
        for j in range(i + 1, N):
            M[i][j] = -L * A[j] * K(X[i], X[j])
            M[j][i] = -L * A[i] * K(X[j], X[i])
    y = np.linalg.solve(M, g)
    u = []
    for x_0 in X:
        u.append( L * sum(A[k] * K(x_0, X[k]) * y[k] for k in range(N)) + f(x_0))
    for i in range(N):
        print("{}\t{}\t{}\n".format(X[i], y[i], u[i]), end='')

In [5]:
def MPP():
    def stop(y_1, y_2):
        for i in range(N):
            if abs(y_1[i] - y_2[i] >= EPS):
                return False
        return True

    Y = [[0 for i in range(N)], [1 for i in range(N)]]
    while not stop(Y[-1], Y[-2]):
        y = scipy.interpolate.lagrange(X, Y[-1])
        Y.append([])
        for i in range(N):
            K_1 = lambda s: K(X[i], s) * y(s)
            Y[-1].append(L * scipy.integrate.quad(K_1, a, b)[0] + f(X[i]))
    for i in range(N):
        print("{}\t{}\n".format(X[i], Y[-1][i]), end='')

In [6]:
def mech_quadr_Volterra():
    a, b = 0, pi/2
    h = (b - a) / N
    K = lambda x, s: sin(x - s)
    L = 1.
    f = lambda x: 1 - cos(x)
    A = [h for i in range(N)]
    A[0] = A[-1] = h/2
    y = [f(X[0]),]
    for i in range(1, N):
        y.append( (f(X[i]) + L*sum(A[k]*K(X[i], X[k])*y[k] for k in range(i)))/(1 - L*A[i]*K(X[i], X[i])) )
    for i in range(N):
        print("{}\t{}\n".format(X[i], y[i]), end='')